In [1]:
from globus_compute_sdk import Client, Executor

# Define Globus Compute client and function UUID
compute_client = Client()
endpoint_uuid = "ae99f53a-1275-48a6-a972-693a80d6c982"

# Define function to run remotely
def tomography_reconstruction(proj_file, flat_file, dark_file, angles_file, recon_init, recon_end, rot_center, output_dir):
    import matplotlib.pyplot as plt
    import os
    import h5py
    import tomopy

    def load_data(proj_file, flat_file, dark_file, angles_file, recon_init, recon_end):
        """
        Loads projection, flat, dark, and angles data from HDF files.
        Slices the projection, flat, and dark data between recon_init and recon_end.
        Converts the rotation angles to radians.
        """
        with h5py.File(proj_file, 'r') as f_proj:
            proj = f_proj['entry']['data']['data'][:, recon_init:recon_end, :]
            print('proj', proj.shape, proj.dtype)
        with h5py.File(flat_file, 'r') as f_flat:
            flats = f_flat['entry']['data']['data'][:, recon_init:recon_end, :]
            print('flats', flats.shape, flats.dtype)
        with h5py.File(dark_file, 'r') as f_dark:
            darks = f_dark['entry']['data']['data'][:, recon_init:recon_end, :]
            print('darks', darks.shape, darks.dtype)
        with h5py.File(angles_file, 'r') as f_angles:
            theta = f_angles['entry']['data']['rotation_angle'][:]
            theta = theta * 3.141592653589793 / 180  # convert to radians
            print('theta', theta.shape, theta.dtype)
        return proj, flats, darks, theta

    def recon_data(proj, flats, darks, theta):
        """
        Normalizes the projection data, finds the rotation center,
        applies logarithmic normalization, reconstructs the tomographic image,
        and applies a circular mask.
        """
        proj_norm = tomopy.normalize(proj, flats, darks)
        print('proj_norm', proj_norm.shape, proj_norm.dtype)

        # Find rotation center from normalized projections
        rot_center = tomopy.find_center_vo(proj_norm)
        print('rot_center', rot_center)

        # Apply log normalization
        proj_log = tomopy.minus_log(proj_norm)
        print('proj_norm_ml', proj_log.shape, proj_log.dtype)

        # Reconstruct using the gridrec algorithm
        recon = tomopy.recon(proj_log, theta, center=rot_center, algorithm='gridrec')
        print('recon', recon.shape)

        # Apply a circular mask to the reconstruction
        recon_masked = tomopy.circ_mask(recon, axis=0, ratio=0.95)

        return recon_masked, proj_norm, rot_center

    def save_data(recon, rot_center, output_dir='recon'):
        """
        Saves the reconstructed volume as a stack of TIFF files and writes the
        rotation center to a text file in the specified output directory.
        """
        import dxchange
        os.makedirs(output_dir, exist_ok=True)

        # Save the reconstruction; files will be written with a prefix in the output directory.
        dxchange.write_tiff_stack(recon, fname=os.path.join(output_dir, 'recon'), axis=0)
        print('Reconstruction data saved with shape:', recon.shape)

        # Save the rotation center to a text file.
        center_file = os.path.join(output_dir, 'center.txt')
        with open(center_file, 'w') as file:
            file.write(str(rot_center))
        print('Rotation center saved:', center_file)

    def save_images(proj, recon, output_dir='.'):
        """
        Saves representative images of the projection and reconstruction data as PNG files.
        """
        os.makedirs(output_dir, exist_ok=True)
        proj_img_path = os.path.join(output_dir, 'proj.png')
        recon_img_path = os.path.join(output_dir, 'recon.png')

        # Save a sinogram (first column of projection data)
        plt.imsave(proj_img_path, proj[:, 0, :], cmap='gray')
        # Save the middle slice of the reconstruction
        mid_slice = recon[recon.shape[0] // 2, :, :]
        plt.imsave(recon_img_path, mid_slice, cmap='gray')

        print(f"Projection image saved as {proj_img_path}")
        print(f"Reconstruction image saved as {recon_img_path}")

    proj, flats, darks, theta = load_data(proj_file, flat_file, dark_file, angles_file, recon_init, recon_end)
    recon, proj_norm, rot_center = recon_data(proj, flats, darks, theta)
    save_data(recon, rot_center, f'{output_dir}/recon_output')
    save_images(proj_norm, recon, f'{output_dir}/recon_output')
    
    return f"Reconstruction completed with rot_center: {rot_center}"


In [ ]:

# Define base path
base_path = '/home/ravescovi/workspace/NSLS2'

# Define data parameters
data = {
    'proj_file': f'{base_path}/scan_00244/proj_00000.hdf',
    'angles_file': f'{base_path}/scan_00244/scan_00244.nxs',
    'dark_file': f'{base_path}/scan_00245/dark_00000.hdf',
    'flat_file': f'{base_path}/scan_00245/flat_00000.hdf',
    'recon_init': 1000,
    'recon_end': 1512,
    'rot_center': None,
    'output_dir': f'{base_path}/compute_test'
}

# Execute function on remote Globus Compute endpoint
with Executor(endpoint_id=endpoint_uuid) as gce:
    future = gce.submit(tomography_reconstruction, **data)
    result = future.result()

print(result)


Reconstruction completed with rot_center: 1609.25


In [ ]:

# Define base path
base_path = '/home/ravescovi/workspace/NSLS2'

# Define data parameters
data = {
    'proj_file': f'{base_path}/scan_00244/proj_00000.hdf',
    'angles_file': f'{base_path}/scan_00244/scan_00244.nxs',
    'dark_file': f'{base_path}/scan_00245/dark_00000.hdf',
    'flat_file': f'{base_path}/scan_00245/flat_00000.hdf',
    'recon_init': 1000,
    'recon_end': 1512,
    'rot_center': None,
    'output_dir': f'{base_path}/test_container'
}


# Define your container URI (modify this with the correct container image)
CONTAINER_URI = "localhost/tomography-container:latest"
# Execute function on remote Globus Compute endpoint using a container
with Executor(endpoint_id=endpoint_uuid) as gce:
    future = gce.submit(
        tomography_reconstruction, 
        **data, 
        ## This will raise an error because it seems globus_compute_sdk does not support user_config
        # user_config={'container_type':'podman',
        #              'container_uri': CONTAINER_URI,
        #              'run_dir':'/home/ravescovi'}  
    )
    result = future.result()

print(result)


TaskExecutionFailed: 
 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 Traceback from attempt: final attempt
 Traceback (most recent call last):
   File "/home/ravescovi/miniconda3/envs/tomo/lib/python3.11/concurrent/futures/_base.py", line 401, in __get_result
     raise self._exception
 parsl.executors.errors.BadStateException: Executor GlobusComputeEngine-HighThroughputExecutor failed due to: Error 1:
 	Job is marked as MISSING since the workers failed to register to the executor. Check the stdout/stderr logs in the submit_scripts directory for more debug information
 	EXIT CODE: 127
 	STDOUT: Found cores : 32
 Launching worker: 1

 	STDERR: Error: runc: runc create failed: unable to start container process: error during container init: exec: "globus-compute-endpoint": executable file not found in $PATH: OCI runtime attempted to invoke a command that was not found



 --------------------------------------------------------------------